In [9]:
import anthropic
from anthropic.types.beta.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.beta.messages.batch_create_params import Request
from config import ANTHROPIC_API_KEY
import time

client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY,
)

try:
    message_batch = client.beta.messages.batches.create(
        betas=["message-batches-2024-09-24"],
        requests=[
            Request(
                custom_id="first-prompt-in-my-batch",
                params=MessageCreateParamsNonStreaming(
                    model="claude-3-5-sonnet-20241022",
                    max_tokens=100,
                    system="You are a dog expert.",
                    messages=[{
                        "role": "user",
                        "content": "Hey Claude, tell me a short fun fact about italian spitz dogs!",
                    }]
                )
            ),
            Request(
                custom_id="second-prompt-in-my-batch",
                params=MessageCreateParamsNonStreaming(
                    model="claude-3-5-sonnet-20240620",
                    max_tokens=100,
                    system="You are a dog expert.",
                    messages=[{
                        "role": "user",
                        "content": "Hey Claude, tell me a short fun fact about italian spitz dogs!",
                    }]
                )
            )
        ]
    )
    print("Batch created:", message_batch)

    # Poll for results
    while True:
        batch_status = client.beta.messages.batches.retrieve(message_batch.id)
        print(f"Batch {batch_status.id} processing status: {batch_status.processing_status}")

        if batch_status.processing_status == "ended":
            # Stream results
            for result in client.beta.messages.batches.results(message_batch.id):
                result_type = result.result.type
                if result_type == "canceled":
                    print(f"\nRequest canceled for {result.custom_id}")
                elif result_type == "errored":
                    print(f"\nError for {result.custom_id}:", result.result.error)
                elif result_type == "expired":
                    print(f"\nRequest expired for {result.custom_id}")
                elif result_type == "succeeded":
                    print(f"\nSuccess for {result.custom_id}:")
                    print(result.result.message.content)
            break

        # Add a small delay between polling attempts
        time.sleep(5)

except anthropic.APIError as e:
    print(f"API Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Batch created: BetaMessageBatch(id='msgbatch_01Gn8FoN6kfJsqDphTC9VFRq', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2024, 10, 29, 19, 14, 44, 205229, tzinfo=datetime.timezone.utc), ended_at=None, expires_at=datetime.datetime(2024, 10, 30, 19, 14, 44, 205229, tzinfo=datetime.timezone.utc), processing_status='in_progress', request_counts=BetaMessageBatchRequestCounts(canceled=0, errored=0, expired=0, processing=2, succeeded=0), results_url=None, type='message_batch')
Batch msgbatch_01Gn8FoN6kfJsqDphTC9VFRq processing status: in_progress
Batch msgbatch_01Gn8FoN6kfJsqDphTC9VFRq processing status: in_progress
Batch msgbatch_01Gn8FoN6kfJsqDphTC9VFRq processing status: in_progress
Batch msgbatch_01Gn8FoN6kfJsqDphTC9VFRq processing status: in_progress
Batch msgbatch_01Gn8FoN6kfJsqDphTC9VFRq processing status: in_progress


KeyboardInterrupt: 